# Azure hackathon

## Import

In [80]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk.corpus import stopwords
stop = stopwords.words('english')

import numpy as np
import pandas as pd

# !pip3 install textblob
# !python3 -m textblob.download_corpora

from textblob import TextBlob


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jayant\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jayant\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Jayant\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Jayant\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Jayant\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
nltk.download()
from nltk.corpus import stopwords
stop = stopwords.words('english')
# stop = list(map(lambda x: x.upper(),stop))
stop.remove('in')
stop.remove('at')
stop.remove('and')
stop.remove('to')


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


## Get article in document

In [3]:
document = """Lower costs and stable pricing came to the rescue of UltraTech, even as the pandemic impacted its March quarter performance. UltraTech Cement reported a 6% year-on-year increase in its consolidated net profit at Rs 1,131 crore for the quarter ended March 31. This is the company’s normalised profit.

The company has reported both normalised and net profit numbers this quarter, on account of deferred tax liabilities because of change in the tax regime from 35% to the new 25%. Normalised profit after tax (PAT) of Rs 1,131 crore is before considering the benefit of Rs 2,112 crore of reversal of deferred tax liabilities as on April 1, 2019, due to change in income tax rates. The company’s reported net profit stood at Rs 3,243 crore, including the gain of deferred tax liabilities (DTLs).

RELATED NEWS
Big 4 auditors have lost credibility among 57% investors: IiAS surveyBig 4 auditors have lost credibility among 57% investors: IiAS surveyDo mutual fund managers justify their fees? Here's how fund alpha compares with management feesDo mutual fund managers justify their fees? Here's how fund alpha compares with management fees

According to HDFC Securities Institutional Research Desk, UltraTech’s volumes suffered (down 16% y-o-y) in the March quarter owing to the Covid-19 impact. However, stable pricing and lower energy cost drove unitary ebitda expansion 14% y-o-y to a solid Rs 1139/MT (up 13% quarter-on-quarter), thus moderating consolidated earnings before interest, taxes, depreciation and amortisation (ebitda) decline to 4% y-o-y. “Subsequently, higher treasury gains and lower tax rate drove up adjusted PAT by 6% y-o-y. During FY20, strong profits (ebitda/apat up 28/51%), along with working capital release and flattish capex boosted free cashflows and its net debt fell 25% y-o-y to `16.9 billion. Net debt/ebitda cooled off to 1.8x vs 3x y-o-y,” the brokerage said.

The consolidated revenue from operations declined 13% y-o-y to Rs 10,579 crore on account of disruptions led by Covid-19 related shutdowns, resulting in lower sales volumes during the quarter and the full year. “In the face of the unprecedented situation arising out of the pandemic, the company’s operations across locations were shut down in line with the government directives. Construction activity across the country was halted, which is normally at its peak in the month of March, leading to an adverse impact on the company’s operations during the quarter ended March 31, 2020,” the company statement said.

Consolidated sales volume suffered a sharp decline of 16% to 21.44 million tonne in Q4FY20, while for the full year volumes fell 4% to 82.33 million tonne.

However, the consolidated ebitda remained flat at Rs 2,639 crore, supported by lower fuel and logistics costs, though, impacted by higher raw material prices. Consequently, consolidated ebitda margins came in at 25%, a rise of 300 basis points against the figures of fourth quarter last year.

Logistics costs declined 3% y-o-y to Rs 1149 per tonne on account of benefit due to exemption of railway busy-season surcharge. The energy costs were lower by 13% y-o-y to Rs 914 per tonne with benefit from lower fuel prices and increased pet coke usage. The raw material costs increased 5% y-o-y to Rs 497 per tonne. This was on account of higher fly ash prices and increased share of new products.

Meanwhile, cement dealers across the country expect significantly sluggish sales, elongated credit period to retailers, and higher working capital needs in the wake of the Covid-19 pandemic this financial year, according to a Crisil survey. Majority of the dealers surveyed said they expected volumes to shrink 10-30% in financial year 2020-2021 in the base case scenario, if the lockdown eases in May. “Extension beyond this can worsen these figures,” rating agency noted.

Cement prices trend in May, to date, indicates that prices have been increased by Rs 20-30 per bag in the North, Central and West markets; Rs 20-50 per bag in the East markets (Rs 20-25 per bag in all states except Bihar where the price hike is Rs 50 per bag), and Rs 40-90 per bag in the south region. “If these prices sustain, price improvement in Q1FY21 should be at 6-16% quarter-on-quarter, much higher than our expectation of 0-1% q-o-q price increase. Current cement prices in the North, Central and South (net of all discounts) markets are at all-time high levels,” analysts at Emkay Research said in a recent report."""

## Process Document

In [60]:
def nltk_process(document):
    document = " ".join([i for i in document.split() if i not in stop])
    sentences = nltk.sent_tokenize(document)
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    return sentences

def nltk_eval(document):
    chunkGram = r"""
    ADDRESS: {<JJ.?|CD.?>+<CD.?|JJ.?|NNP.?>+<CD|NNP>+}
    TEST : {}
    NP: {<DT|JJ|NN.*>+}
    """
    organization = {}
    location = {}
    chunkParser = nltk.RegexpParser(chunkGram)
    for tagged_sentence in nltk_process(document):
            for chunk in nltk.ne_chunk(tagged_sentence):
                if type(chunk) == nltk.tree.Tree:
                    if chunk.label() == 'ORGANIZATION':
                        ## Organization is found
                        temp = (' '.join([c[0] for c in chunk])).lower()
                        if temp in organization:
                            organization[temp] += 1
                        else:
                            organization[temp] = 1
                    if chunk.label() == 'GPE':
                        temp = ' '.join([c[0] for c in chunk])
                        if temp in location:
                            location[temp] += 1
                        else:
                            location[temp] = 1
    organization = {k: v for k, v in sorted(organization.items(), key=lambda item: item[1], reverse=True)}
    location = {k: v for k, v in sorted(location.items(), key=lambda item: item[1], reverse=True)}
    
    print("Organizations:\n",list(organization.keys()))
    print("Location:\n",list(location.keys()))
                            
    return organization, location
    # return names, organization, location, address, other

## Setting up one time company and subsector loading

### Utilities function

In [54]:
def clean_companies(companies):
    cleaned_companies = []
    for c in companies:
        x = c.lower()
        if x.endswith('limited'):
            x = x[:-8]
        elif x.endswith('ltd'):
            x = x[:-4]
        elif x.endswith('limited.'):
            x = x[:-9]
        elif x.endswith('ltd.'):
            x = x[:-5]
        else:
            print(x)
        cleaned_companies.append(x)
    return cleaned_companies

def clean_subsectors(subsector):
    return subsector.str.lower()
    

### Old

In [42]:
df = pd.read_csv('EQUITY_L.csv')
df.head()

,SYMBOL,NAME OF COMPANY,SERIES,DATE OF LISTING,PAID UP VALUE,MARKET LOT,ISIN NUMBER,FACE VALUE
0,20MICRONS,20 Microns Limited,EQ,06-OCT-2008,5,1,INE144J01027,5
1,21STCENMGM,21st Century Management Services Limited,EQ,03-MAY-1995,10,1,INE253B01015,10
2,3IINFOTECH,3i Infotech Limited,EQ,22-APR-2005,10,1,INE748C01020,10
3,3MINDIA,3M India Limited,EQ,13-AUG-2004,10,1,INE470A01017,10
4,3PLAND,3P Land Holdings Limited,EQ,19-JUL-1995,2,1,INE105C01023,2


In [43]:
companies = list(df['NAME OF COMPANY'].values)
cleaned_companies = clean_companies(companies)
print(cleaned_companies)

bank of baroda
bank of india
canara bank
central bank of india
general insurance corporation of india
indian bank
indian overseas bank
bank of maharashtra
punjab national bank
punjab & sind bank
state bank of india
uco bank
union bank of india
['20 microns', '21st century management services', '3i infotech', '3m india', '3p land holdings', '5paisa capital', '63 moons technologies', '8k miles software services', 'a2z infra engineering', 'aarti drugs', 'aarti industries', 'aarvee denims & exports', 'aavas financiers', 'aban offshore', 'abb india', 'abbott india', 'aditya birla capital', 'aditya birla fashion and retail', 'abm international', 'acc', 'accelya solutions india', 'action construction equipment', 'adani enterprises', 'adani gas', 'adani green energy', 'adani ports and special economic zone', 'adani power', 'adani transmission', 'adf foods', 'adhunik industries', 'ador welding', 'adroit infotech', 'allied digital services', 'advani hotels & resorts (india)', 'advanced enzyme te

### New

In [69]:
df = pd.read_csv('finalBetaDB.csv')
df.rename(columns={' Sector': 'Sector', ' Sub-sector':'Subsector'}, inplace=True)
df = df.replace(np.nan, '', regex=True)
df.head()

,Symbol,Sector,Subsector,Company,beta
0,20MICRONS,Basic Materials,Other Industrial Metals & Mining,20 Microns Limited,0.783931
1,3IINFOTECH,Technology,Information Technology Services,3i Infotech Limited,1.167706
2,3MINDIA,Industrials,Conglomerates,3M India Limited,0.389235
3,8KMILES,Technology,Software-Application,8K Miles Software Services Limited,1.102964
4,A2ZINFRA,Industrials,Engineering & Construction,A2Z Infra Engineering Limited,0.549921


In [70]:
cols = ['Sector', 'Subsector']
for col in cols:
    df[col] = df[col].str.strip()

In [71]:
cleaned_companies = clean_companies(df['Company'])
cleaned_subsectors = clean_subsectors(df['Subsector'])
# print(cleaned_companies)
print(cleaned_subsectors)

bank of baroda
bank of india
canara bank
central bank of india
general insurance corporation of india
indian bank
indian overseas bank
bank of maharashtra
punjab national bank
punjab & sind bank
state bank of india
uco bank
union bank of india
0             other industrial metals & mining
1              information technology services
2                                conglomerates
3                         software-application
4                   engineering & construction
5       drug manufacturers-specialty & generic
6                          specialty chemicals
7                        textile manufacturing
8                           oil & gas drilling
9               specialty industrial machinery
10                  drug manufacturers-general
11                     financial conglomerates
12                       apparel manufacturing
13                         specialty chemicals
14                          building materials
15             information technology services
16  

## Finding sentiment

In [83]:
organization, _ = nltk_eval(document)
print(organization)

{'ultratech': 3, 'pat': 2, 'cement': 1, 'dtls': 1, 'related': 1, 'news': 1, 'iias': 1, 'feesdo': 1, 'hdfc securities institutional': 1, 'fy20': 1, 'q4fy20': 1, 'central west': 1, 'q1fy21': 1, 'central south': 1}


In [97]:
def find_subsectors(organizations, cleaned_subsectors):
    """Find subsectors in organization"""
    
    organ_to_subsector = {}
    for subsector in cleaned_subsectors:
        for organization in organizations.keys():
            if organization in subsector.split(' '):
                if organization not in organ_to_subsector:
                    organ_to_subsector[organization] = list()
                organ_to_subsector[organization].append(subsector)
    return organ_to_subsector

def find_companies(organizations, cleaned_companies):
    
    organ_to_company = {}
    for company in cleaned_companies:
        for organization in organizations.keys():
            if organization in company.split(' '):
                if organization not in organ_to_company:
                    organ_to_company[organization] = list()
                organ_to_company[organization].append(company)
    return organ_to_company

def find_organ_context(document, organization):
    count_sentences = 0
    organ_to_sentenceid = {}
    for sentence in document.split('.'):
        count_sentences += 1
        for organ in organization.keys():
            if organ in sentence.lower():
                if organ not in organ_to_sentenceid:
                    organ_to_sentenceid[organ] = list()
                organ_to_sentenceid[organ].append(count_sentences)
    return organ_to_sentenceid, count_sentences

def remove_other_organization(organizations, organ_to_subsector, organ_to_company):
    dict_organizations = organizations.copy()
    for organization in organizations.keys():
        if organization not in (organ_to_subsector.keys() or organ_to_company.keys()):
            dict_organizations.pop(organization)
    return dict_organizations

In [82]:
organ_to_subsector = find_subsectors(organization, cleaned_subsectors)
organ_to_company = find_companies(organization, cleaned_companies)

In [84]:
s = TextBlob(document)
s.sentiment

Sentiment(polarity=0.0697366143957053, subjectivity=0.37840785477149114)

In [98]:
updated_organizatation = remove_other_organization(organization, organ_to_subsector, organ_to_company)
updated_organizatation

{'ultratech': 3, 'cement': 1}

In [99]:
organ_to_sentenceid, total_sentences = find_organ_context(document, updated_organizatation)
organ_to_sentenceid

{'ultratech': [1, 2, 7], 'cement': [2, 26, 29, 31]}

In [104]:
def find_sentiment_of_context(document, organ_to_sentenceid, total_sentences):
    list_of_sentences = document.split('.')
    polarity_of_organ = {}

    for item, value in organ_to_sentenceid.items():
        polarity_of_organ[item] = list()
        for i in value:
            lower_value = i-1
            upper_value = i+1
            if lower_value < 0:
                lower_value = 0
            if upper_value > total_sentences:
                upper_value = total_sentences
            current_context = str(list_of_sentences[lower_value:upper_value+1])
            current_context = TextBlob(current_context)
            polarity_of_organ[item].append(current_context.sentiment.polarity)

    print(polarity_of_organ)
    return polarity_of_organ

polarity_of_organ = find_sentiment_of_context(document, organ_to_sentenceid, total_sentences)

{'ultratech': [0.0, 0.04545454545454545, 0.03148148148148147], 'cement': [0.04545454545454545, -0.04583333333333334, 0.05857142857142857, 0.032]}


## Test news extractor

In [18]:
import requests
import urllib.parse
url = "https://www.bloombergquint.com/coronavirus-outbreak/vaccine-from-moderna-shows-early-signs-of-viral-immune-response"
response  = requests.get(url)

nltk_eval(response.text)

Organizations:
 ['completeurl', 'bloombergquint', 'top__28wgm', 'bq blue', 'union budget', 'union', 'coronavirus vaccine', 'rbi', 'fb8e1e', 'fineprint', 'elections', 'center__luymc', 'linkedin', 'adunit', 'gst', 'ipos', 'pwaeventdate', 'coronavirus outbreak', 'translatex', 'coronavirus pandemic', 'icici lombard', 'deferredprompt', 'scripttag', 'df0101', 'ff6564', 'corporate', 'nbcc', 'mrf', 'pgti', 'csr journal', 'mutual fund', 'news', 'markets', 'dell', 'rpg', 'mcx', 'lic', 'embassy group', 'hdfc', 'hdfc life', 'pti', 'telangana assembly election', 'mitsubishi electric', 'iifl', 'cfa', 'barriers', 'cleartax', 'real estate', 'real economy check', 'bank baroda', 'intel', 'pwadiff', 'of viral', 'bloomberg', 'moderna', 'eventmethod', 'eventname', 'optinmonsterapp', 'quint', 'business', 'whatsapp', 'americanexpress', 'bgbs', 'pnb fraud', 'bqportfolio videos', 'getdiffindays', 'addtohomescreen', 'parseint', 'doctype', 'moderna inc.', 'university oxford', 'mrna', 'rna', 'u.s. national instit

'Success'

In [19]:
response.json

<bound method Response.json of <Response [200]>>

In [45]:
import lxml.html

In [65]:
url = "https://www.bloombergquint.com/coronavirus-outbreak/vaccine-from-moderna-shows-early-signs-of-viral-immune-response"
response  = requests.get(url)
doc = lxml.html.fromstring(response.content)
tags = doc.xpath('//*[@class="story-element story-element-text"]')
total_tags = []
for tag in tags:
    total_tags.append(tag.text_content())
tags

[<Element div at 0x2dcfedd5598>]

In [72]:
import lxml.etree
root = lxml.etree.fromstring(response.content)
for elem in root.xpath('//*[@class="story-element story-element-text"]'):
    print("j")

XMLSyntaxError: EntityRef: expecting ';', line 4, column 2589 (<string>, line 4)